# Data Scrapping CA 2- Kenneth Xavier Dsilva ID: 10600644


* I have scrapped two websites IMDB and OMDB 
* OMDB is an API which i used to get the movie details from, IMDB i have scrapped to get the Titles of the Top 1000 movies from.
* This i have then converted to a DataFrame and exported as a CSV.



In [1]:
import requests 
from pprint import PrettyPrinter
import json
import pandas as pd
from bs4 import BeautifulSoup
pp= PrettyPrinter()
#from google.colab import files

In [3]:
#Getting list of Movie Names
movie_titles=[]
url = "https://www.imdb.com/list/ls055592025/"
response1 = requests.get(url)
soup = BeautifulSoup(response1.content,'html.parser')

title_data = soup.findAll('div', attrs={'class':'lister-item mode-advanced'})

print('.')
for store in title_data:
    name = store.h3.a.text
    movie_titles.append(name)

.


In [4]:
movie_titles=[]
url = "https://www.imdb.com/list/ls055592025/"
response1 = requests.get(url)
soup = BeautifulSoup(response1.content,'html.parser')

title_data = soup.findAll('div', attrs={'class':"lister-item-content"})

print('.')

for store in title_data:
    name = store.h3.a.text
    movie_titles.append(name)

.


In [5]:
len(movie_titles)

100

In [6]:
import requests

movie_name = []
year = []
actors = []
directors = []
writer = []
rating=[]

titles=[]
count=1
for name in movie_titles:
  titles.append(name)


for title in titles:
  url = 'http://www.omdbapi.com/?apikey=9fe2dc32&r=json&t='+title

  resp = requests.get(url)
  data = resp.content
  h = json.loads(data)
  if(str((h["Response"]))=='False'):
    print('{} isint available due to {}'.format(title, str((h["Error"]))))
  else:
    
    title=str((h["Title"]))
    title = title.replace('[','').replace(']','').replace("'","")
    movie_name.append(title)
    #else: print("No Movie")
  #Year
    y=str((h["Released"]))
    y=y.replace('[','').replace(']','').replace("'","").replace(" ",'/')
    y =y.replace('Jan','01').replace('Feb','02').replace('Mar','03').replace('Apr','04').replace('May','05').replace('Jun','06').replace('Jul','07').replace('Aug','08').replace('Sep','09').replace('Oct','10').replace('Nov','11').replace('Dec','12')
    year.append(y)

  #Actor
    a = str((h["Actors"]))
    a= a.replace('[','').replace(']','').replace("'","").replace('"','')
    actors.append(a)

  #Directors
    d = str((h["Director"]))
    d= d.replace('[','').replace(']','').replace("'","").replace('"','')
    directors.append(d)

  #Writer
    w = str((h["Writer"]))
    w= w.replace('[','').replace(']','').replace("'","").replace('"','')
    writer.append(w)


  #Rating
    rate= str((h["Ratings"]))
    rate=rate.replace("[{'Source': 'Internet Movie Database', 'Value': ", "").replace("}]","").replace("'","")
    #print(rate)
    rate=rate[0:3]
    rating.append(rate)
  df= pd.DataFrame({"Movie Name":movie_name,
                    "Year":year,
                    "Actors": actors,
                    "Directors":directors,
                    "Writer": writer,
                    "Rating":rating
      })

Il buono, il brutto, il cattivo isint available due to Movie not found!


In [7]:
len(df)

99

In [8]:
df.head()

,Movie Name,Year,Actors,Directors,Writer,Rating
0,The Godfather,24/03/1972,"Marlon Brando, Al Pacino, James Caan",Francis Ford Coppola,"Mario Puzo, Francis Ford Coppola",9.2
1,The Shawshank Redemption,14/10/1994,"Tim Robbins, Morgan Freeman, Bob Gunton",Frank Darabont,"Stephen King, Frank Darabont",9.3
2,Schindlers List,04/02/1994,"Liam Neeson, Ralph Fiennes, Ben Kingsley",Steven Spielberg,"Thomas Keneally, Steven Zaillian",8.9
3,Raging Bull,19/12/1980,"Robert De Niro, Cathy Moriarty, Joe Pesci",Martin Scorsese,"Jake LaMotta, Joseph Carter, Peter Savage",8.2
4,Casablanca,23/01/1943,"Humphrey Bogart, Ingrid Bergman, Paul Henreid",Michael Curtiz,"Julius J. Epstein, Philip G. Epstein, Howard Koch",8.5


#

In [9]:
from py2neo import Graph
graph = Graph("bolt://localhost:11010", auth=("neo4j", "neo")) 

In [10]:
from neo4j import GraphDatabase

movie_list = df
movie_list = movie_list.replace(" ","")
movie_list = movie_list.values.tolist()

exe_comds_Movie = []
exe_comds_Actor = []
exe_comds_Director = []
exe_comds_Writer = []
exe_comds_Rating = []

for i in movie_list:
    
    create_stat ="CREATE(m:Movie_name{Movie:'" +str(str(i[0]))+"'})"
    create_stat_ActorName ="CREATE(a:Actor_Names{Actor: '"+ str(i[2]) +"'})"
    create_stat_Directors ="CREATE(d:Director_Names{Director: '"+ str(i[3]) +"'})" 
    create_stat_Writers = "CREATE(w:Writer_Name{Writer:'"+ str(i[4]) +"'})"
    create_stat_Rating = "CREATE(r:Rating{Rating:'"+ str(i[5]) +"'})"                                   
    
    exe_comds_Movie.append(create_stat)
    exe_comds_Actor.append(create_stat_ActorName)
    exe_comds_Director.append(create_stat_Directors)
    exe_comds_Writer.append(create_stat_Writers)
    exe_comds_Rating.append(create_stat_Rating)
                                                         
                                                         
#def exec_tran(exe_comds_Movie,exe_comds_Actor,exe_comds_Director,exe_comds_Writer,exe_comds_Rating):
def exec_tran(exe_comds_Movie,exe_comds_Actor,exe_comds_Director,exe_comds_Writer,exe_comds_Rating):
    graph = GraphDatabase.driver(uri="bolt://localhost:11010", auth=("neo4j", "neo")) 
    sessions=graph.session()
                                                         
                                                         
    for i in exe_comds_Movie:
        sessions.run(i)
    for i in exe_comds_Actor:
        sessions.run(i)
    for i in exe_comds_Director:
        sessions.run(i)
    for i in exe_comds_Writer:
        sessions.run(i)
    #for i in exe_comds_Rating:
        sessions.run(i)

                                                         
                                                         
exec_tran(exe_comds_Movie,exe_comds_Actor,exe_comds_Director,exe_comds_Writer,exe_comds_Rating)


In [11]:
result = {"label": [], "count": []}
for label in graph.run("CALL db.labels()").to_series():
    query = f"MATCH (:`{label}`) RETURN count(*) as count"
    count = graph.run(query).to_data_frame().iloc[0]['count']
    result["label"].append(label)
    result["count"].append(count)
nodes_df = pd.DataFrame(data=result)
nodes_df.sort_values("count")

,label,count
0,Movie,198


In [12]:
graph.run("CALL db.schema.visualization()").data()

[{'nodes': [Node('Movie', constraints=[], indexes=[], name='Movie')],
  'relationships': []}]

This is the Output in Neo4J


<div align="center">
    <img src="https://user-images.githubusercontent.com/54445900/163680788-d276c860-2c12-440c-b8c9-218ea27005c4.png" alt="Neo4j Nodes"/>
</div>

## Creating Relationship between movie, Directors and Actors

In [13]:
Dir_Movie_Act_rel = """
create (m:Movie)<-[:Directed]-(d:Director)-[:Actors_Directed]->(a:Actor)
RETURN m
"""

Movie_Writer_rel ="""
CREATE (m:Movie)-[:Writer]->(w:Writers)
Return m
"""

Movie_Rating_rel ="""
CREATE (m:Movie)-[:Rated]->(r:Rating)
Return m
"""

Actor_Movie_rel ="""
CREATE (a:Actor)-[:Cast_Of]->(m:Movie)
Return m
"""
graphdri = GraphDatabase.driver(uri="bolt://localhost:11010", auth=("neo4j", "neo")) 
sessions=graphdri.session()
sessions.run(Dir_Movie_Act_rel)
sessions.run(Movie_Writer_rel)
sessions.run(Movie_Rating_rel)
sessions.run(Actor_Movie_rel)

In [14]:
graph.run("CALL db.schema.visualization()").data()

[{'nodes': [Node('Movie', constraints=[], indexes=[], name='Movie'),
   Node('Writers', constraints=[], indexes=[], name='Writers'),
   Node('Actor', constraints=[], indexes=[], name='Actor'),
   Node('Rating', constraints=[], indexes=[], name='Rating'),
   Node('Director', constraints=[], indexes=[], name='Director')],
  'relationships': [Rated(Node(), Node()),
   Cast_Of(Node(), Node()),
   Actors_Directed(Node(), Node()),
   Directed(Node(), Node()),
   Writer(Node(), Node())]}]

If you Run this in Neo4j this is what you get:
<div align="left">
    <img src="https://user-images.githubusercontent.com/54445900/163680832-72ab9049-7438-41a5-82e5-313b1a7720ec.png" alt="Neo4j Schema"/>
</div>



In [16]:
graph = Graph("bolt://localhost:11010", auth=("neo4j", "neo")) 
query = "Match (n) return (n)"
result = graph.query(query)
result.data()

[{'n': Node('Movie', Actor_Names='Julie Andrews, Christopher Plummer, Eleanor Parker', Director='Robert Wise', Rating='8.0', Title='The Sound of Music', Writer='George Hurdalek, Howard Lindsay, Russel Crouse')},
 {'n': Node('Movie', Actor_Names='Henry Fonda, Lee J. Cobb, Martin Balsam', Director='Sidney Lumet', Rating='9.0', Title='12 Angry Men', Writer='Reginald Rose')},
 {'n': Node('Movie', Actor_Names='Natalie Wood, George Chakiris, Richard Beymer', Director='Jerome Robbins, Robert Wise', Rating='7.5', Title='West Side Story', Writer='Ernest Lehman, Arthur Laurents, Jerome Robbins')},
 {'n': Node('Movie', Actor_Names='Mark Hamill, Harrison Ford, Carrie Fisher', Director='George Lucas', Rating='8.6', Title='Star Wars', Writer='George Lucas')},
 {'n': Node('Movie', Actor_Names='Keir Dullea, Gary Lockwood, William Sylvester', Director='Stanley Kubrick', Rating='8.3', Title='2001: A Space Odyssey', Writer='Stanley Kubrick, Arthur C. Clarke')},
 {'n': Node('Movie', Actor_Names='Henry Tho

In [18]:
query2='''Match (m:Movie) where m.Rating > "9.0"
    Return (m.Director)'''

result=graph.query(query2)
result.data()

[{'(m.Director)': 'Francis Ford Coppola'},
 {'(m.Director)': 'Frank Darabont'},
 {'(m.Director)': 'Francis Ford Coppola'},
 {'(m.Director)': 'Frank Darabont'}]

In [28]:
query3='''Match (m:Movie)
    Return m'''

result=graph.query(query3)
result.data()

[{'m': Node('Movie', Actor_Names='Julie Andrews, Christopher Plummer, Eleanor Parker', Director='Robert Wise', Rating='8.0', Title='The Sound of Music', Writer='George Hurdalek, Howard Lindsay, Russel Crouse')},
 {'m': Node('Movie', Actor_Names='Henry Fonda, Lee J. Cobb, Martin Balsam', Director='Sidney Lumet', Rating='9.0', Title='12 Angry Men', Writer='Reginald Rose')},
 {'m': Node('Movie', Actor_Names='Natalie Wood, George Chakiris, Richard Beymer', Director='Jerome Robbins, Robert Wise', Rating='7.5', Title='West Side Story', Writer='Ernest Lehman, Arthur Laurents, Jerome Robbins')},
 {'m': Node('Movie', Actor_Names='Mark Hamill, Harrison Ford, Carrie Fisher', Director='George Lucas', Rating='8.6', Title='Star Wars', Writer='George Lucas')},
 {'m': Node('Movie', Actor_Names='Keir Dullea, Gary Lockwood, William Sylvester', Director='Stanley Kubrick', Rating='8.3', Title='2001: A Space Odyssey', Writer='Stanley Kubrick, Arthur C. Clarke')},
 {'m': Node('Movie', Actor_Names='Henry Tho